In [5]:
import polars as pl

In [6]:
import matplotlib.pyplot as plt
import json

In [24]:
client = pl.read_csv("../../dataset/client.csv")
electricity_prices = pl.read_csv("../../dataset/electricity_prices.csv")
forecast_weather = pl.read_csv("../../dataset/forecast_weather.csv")
gas_prices = pl.read_csv("../../dataset/gas_prices.csv")
historical_weather = pl.read_csv("../../dataset/historical_weather.csv")
train = pl.read_csv("../../dataset/train.csv")
weather_station_to_county_mapping = pl.read_csv("../../dataset/weather_station_to_county_mapping.csv")
county_id_to_name_map = json.load(open("../../dataset/county_id_to_name_map.json"))
print("finished reading data")

finished reading data


In [25]:
forecast_weather = (
    forecast_weather
    .with_columns(
        pl.col('forecast_datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
    )
)
    
historical_weather = (
    historical_weather
    .with_columns(
        pl.col("datetime") + pl.duration(hours=37)
    )
)

ComputeError: expected Datetime, got str

Error originated just after this operation:
DF ["latitude", "longitude", "origin_datetime", "hours_ahead"]; PROJECT */18 COLUMNS; SELECTION: "None"

In [20]:
historical_weather[["datetime","temperature"]].join(forecast_weather[["forecast_datetime","temperature"]],left_on="datetime",right_on="forecast_datetime")

datetime,temperature,temperature_right
str,f64,f64


In [23]:
electricity_prices

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [24]:
pd.merge(left=train,right=electricity_prices,left_on="datetime",right_on="forecast_date")

,county,is_business,product_type,target,is_consumption,datetime,data_block_id_x,row_id,prediction_unit_id,forecast_date,euros_per_mwh,origin_date,data_block_id_y
0,11,1,0,103.276,1,2023-05-27 11:00:00,633,2004253,67,2023-05-27 11:00:00,1.48,2023-05-26 11:00:00,634
1,0,1,0,539.014,0,2023-05-27 11:00:00,633,2004162,3,2023-05-27 11:00:00,1.48,2023-05-26 11:00:00,634
2,4,0,1,7.564,1,2023-05-27 11:00:00,633,2004193,15,2023-05-27 11:00:00,1.48,2023-05-26 11:00:00,634
3,4,1,3,283.901,1,2023-05-27 11:00:00,633,2004201,18,2023-05-27 11:00:00,1.48,2023-05-26 11:00:00,634
4,6,1,3,170.774,0,2023-05-27 11:00:00,633,2004212,24,2023-05-27 11:00:00,1.48,2023-05-26 11:00:00,634
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99848,7,1,0,816.580,1,2022-11-18 08:00:00,443,1396049,28,2022-11-18 08:00:00,281.17,2022-11-17 08:00:00,444
99849,15,1,1,82.269,1,2021-09-29 07:00:00,28,82957,59,2021-09-29 07:00:00,186.46,2021-09-28 07:00:00,29
99850,4,0,1,61.588,0,2022-03-10 14:00:00,190,582300,15,2022-03-10 14:00:00,49.71,2022-03-09 14:00:00,191
99851,15,0,3,2.616,0,2021-09-30 22:00:00,29,87712,58,2021-09-30 22:00:00,104.80,2021-09-29 22:00:00,30
